In [20]:
import os
import pandas as pd
import numpy as np
from typing import List
from sklearn.preprocessing import PowerTransformer
from scipy.stats import skew, shapiro

pd.set_option('display.max_rows', 500)

os.chdir("C:/Users/jerem/OneDrive/Documents/rocketleague_ml")
file_name = "features"

processed_file = f"data/features/{file_name}.csv"
features = pd.read_csv(processed_file, low_memory=False) # type: ignore
features.columns.values

array(['Fifty-Fifty Touch Percentage', 'Towards Goal Touch Percentage',
       'Towards Teammate Touch Percentage',
       'Towards Opponent Touch Percentage',
       'Towards Open Space Touch Percentage',
       'Percent Time while Closest to Ball',
       'Average Stint while Closest to Ball',
       'Percent Time while Farthest from Ball',
       'Average Stint while Farthest from Ball',
       'Average Distance to Ball', 'Average Distance to Teammates',
       'Average Distance to Opponents', 'Percent Time In Offensive Half',
       'Average Stint In Offensive Half',
       'Percent Time In Defensive Half',
       'Average Stint In Defensive Half', 'Percent Time In Left Half',
       'Average Stint In Left Half', 'Percent Time In Right Half',
       'Average Stint In Right Half', 'Percent Time In Highest Half',
       'Average Stint In Highest Half', 'Percent Time In Lowest Half',
       'Average Stint In Lowest Half', 'Percent Time In Offensive Third',
       'Average Stint In Off

In [21]:
# Define spatial keywords by dimension
field_zones = ["Offensive", "Neutral", "Defensive"]
field_lateral = ["Left", "Middle", "Right"]
field_height = ["Lowest", "Middle-Aerial", "Highest"]

def count_keywords(name: str, keywords: List[str]):
    return sum(1 for k in keywords if k in name)

def is_nested_spatial_col(name: str):
    # Count how many distinct dimensions appear
    zone_hits = count_keywords(name, field_zones)
    lat_hits = count_keywords(name, field_lateral)
    height_hits = count_keywords(name, field_height)
    total_hits = zone_hits + lat_hits + height_hits
    
    # Drop if column name contains 2+ of these components
    return total_hits >= 2 and "Third" in name  # only apply to positional "Third" metrics

drop_nested_spatial = [c for c in features.columns if is_nested_spatial_col(c)]

drop_features = [
    # --- Redundant "Average Stint ..." versions (keep only Percent Time) ---
    *[c for c in features.columns if "Average Stint" in c],

    # --- Team-level or opponent-level fields ---
    *[c for c in features.columns if any(x in c for x in [
        "Team in", "Team With", "Team with", "Opponent Team", "Percentage Blue", "Percentage Orange"
    ])],

    # --- Aggregated spatial layers (too coarse or sums of others) ---
    *[c for c in features.columns if any(x in c for x in [
        "Half"
    ])],

    # --- Nested redundant thirds (we keep basic Third layers, not all cross-combos) ---
    *drop_nested_spatial,

    # --- Rotations: drop one full system (we’ll keep "Full" and drop "Simple") ---
    *[c for c in features.columns if "Simple" in c],

    # --- Speed bins: keep Slow, Boost Speed, Supersonic, Average Speed only ---
    *[c for c in features.columns if any(x in c for x in [
        "Semi-", "Medium-Speed",
    ]) and "Speed" not in c],

    # --- Boost ranges: drop all detailed bin segments ---
    *[c for c in features.columns if any(x in c for x in [
        # "<=25", ">25", "<= 50", ">50", "<= 75", ">75", "<= 100"
    ])],

    # --- Redundant Boost Efficiency subtypes (keep just "Boost Efficiency" and maybe "Far From Ball") ---
    *[c for c in features.columns if any(x in c for x in [
        "Supersonic Speed Boost Efficiency",
        "Drive to Boost Speed Boost Efficiency",
        "Simple Boost Efficiency"
    ])],

    # --- Scored/outcome metrics (not behavior) ---
    "Scored Goal",
    "Team Scored Goal",
]
features = features.drop(columns=drop_features, errors='ignore')

In [22]:
results = []
for col in features.columns:
    if pd.api.types.is_numeric_dtype(features[col]):
        data = features[col].dropna()
        s = skew(data)

        # Skip if not numeric or near-symmetric
        if np.isnan(s) or abs(s) < 0.5:
            continue
        # Shapiro-Wilk test: tests normality (p < 0.05 means NOT normal)
        stat, p = shapiro(data.sample(min(len(data), 5000), random_state=42))  # cap sample size
        results.append({
            "feature": col,
            "skewness": s,
            "shapiro_p": p
        })

skew_features = pd.DataFrame(results).sort_values("skewness", ascending=False)
skew_features

C:\Users\jerem\AppData\Local\Temp\ipykernel_5584\3324221534.py:5: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  s = skew(data)
c:\Users\jerem\OneDrive\Documents\rocketleague_ml\.venv\lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


,feature,skewness,shapiro_p
36,Boost Efficiency,21.828044,1.988141e-45
35,Far From Ball Boost Efficiency,21.744522,2.039253e-45
25,Percent Time While Boosting,14.832230,9.724063e-45
31,Percent Time With Full Boost,11.551655,5.872602e-44
43,Percent Rotating From Full 3 to 1,8.756227,2.778962e-43
19,Percent Time while Stationary,7.720916,2.422384e-42
41,Percent Rotating From Full 1 to 3,7.327582,3.442823e-42
18,Percent Time In Opponents Goal,6.971626,1.376906e-40
20,Percent Time while Slow,6.266241,5.412251e-38
17,Percent Time In Own Goal,4.921164,4.207525e-37


In [26]:
# Make a working copy
skewed_features = features.copy()

records = []  # to collect before/after skewness

for col in skewed_features.select_dtypes(include=["float", "int"]).columns:
    data = skewed_features[col].dropna() * 100
    if len(data) == 0:
        continue

    s_before = skew(data)

    # Skip if not numeric or near-symmetric
    if np.isnan(s_before) or abs(s_before) < 0.5:
        continue

    # Special-case extreme zero-heavy fields
    special_cols = ["Boost Efficiency", "Far From Ball Boost Efficiency", "Percent Time while Slow"]
    if col in special_cols:
        x = np.log1p(data * 100)
        pt = PowerTransformer(method="yeo-johnson")
        data = pt.fit_transform(x.values.reshape(-1, 1)).flatten()
        method = "log1p+yeo-johnson"
    else:
        # Apply transform depending on skew direction
        if s_before > 1:
            # Heavily right-skewed
            if (data <= 1).all() and (data == 0).any():
                # Zero-heavy proportion → Yeo-Johnson
                pt = PowerTransformer(method='yeo-johnson')
                data = pt.fit_transform(data).flatten()
                method = "yeo-johnson"
            else:
                # Count-like → log1p
                data = np.log1p(data)
                method = "log1p"
        elif s_before > 0.5:
            data = np.sqrt(data)
            method = "sqrt"
        elif s_before < -0.5:
            # left-skewed — use square or exp to pull tail right
            data = np.power(data, 2)
            method = "square"
        else:
            method = "none"

    s_after = skew(data)

    # if np.isnan(s_after) or abs(s_after) < 0.5:
    #     continue

    records.append({
        "feature": col,
        "skew_before": s_before,
        "skew_after": s_after,
        "transform": method
    })

# Create comparison table
skew_compare = pd.DataFrame(records).sort_values("skew_after", ascending=False).reset_index(drop=True)
skew_compare


C:\Users\jerem\AppData\Local\Temp\ipykernel_5584\3949755122.py:11: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  s_before = skew(data)


,feature,skew_before,skew_after,transform
0,Percent Time While Boosting,14.832230,6.137107,log1p
1,Percent Time With Full Boost,11.551655,5.544611,log1p
2,Percent Rotating From Full 3 to 1,8.756227,4.488561,log1p
3,Percent Rotating From Full 1 to 3,7.327582,3.715727,log1p
4,Percent Time In Opponents Goal,6.971626,2.888428,log1p
5,Percent Time while Stationary,7.720916,2.398218,log1p
6,Boost Efficiency,21.828044,2.194770,log1p+yeo-johnson
7,Far From Ball Boost Efficiency,21.744522,1.992210,log1p+yeo-johnson
8,Percent Time On Front Wall,3.109558,1.815974,log1p
9,Average Overfill,1.788854,1.788854,log1p


In [24]:
features[skew_compare["feature"]].T

,0,1,2,3,4,5,6,7,8,9,...,524,525,526,527,528,529,530,531,532,533
Boost Efficiency,5.297655e-31,1.313748e-14,9.877749e-07,2.728388e-11,6.149588e-32,0.223389,7.957653e-29,2.569619e-11,0.000000,2.139881e-10,...,4.461370e-21,6.571524e-07,4.958158e-43,5.215143e-17,1.288813e-12,0.000009,2.235776e-41,1.323633e-13,1.286580e-13,0.000026
Far From Ball Boost Efficiency,5.297655e-31,1.097304e-12,4.317803e-06,3.473368e-09,2.279976e-28,0.248210,8.461438e-25,7.483562e-10,0.000000,1.154807e-09,...,8.394856e-21,6.571524e-07,7.913064e-40,4.355929e-15,1.030164e-06,0.000201,1.559759e-37,2.017426e-13,1.792149e-12,0.000026
Percent Time while Slow,6.736460e-02,9.334154e-02,3.487447e-02,1.211867e-01,5.672987e-02,0.144721,3.955501e-02,4.104517e-02,0.084290,7.493664e-03,...,1.345145e-01,2.275763e-02,7.894870e-02,3.723911e-02,2.131147e-02,0.022758,7.003195e-02,1.326241e-01,8.970940e-02,0.034523
Percent Time While Boosting,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
Percent Rotating From Full 3 to 1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.818182e-01,1.250000e-01,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,8.888889e-02,0.000000e+00,0.000000e+00,0.000000,2.941176e-02,0.000000e+00,0.000000e+00,0.000000
Percent Time With Full Boost,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.783905e-02,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,8.099389e-03,0.000000
Percent Rotating From Full 1 to 3,0.000000e+00,6.666667e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.470588e-01,7.142857e-02,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,7.317073e-02,0.000000e+00,0.000000e+00,0.000000,4.651163e-02,0.000000e+00,0.000000e+00,0.000000
Percent Time In Opponents Goal,0.000000e+00,6.922888e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,1.122164e-03,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,8.071303e-03,0.000000
Percent Time while Stationary,1.113780e-03,6.139815e-02,0.000000e+00,0.000000e+00,2.548313e-03,0.000000,0.000000e+00,1.439830e-03,0.000000,0.000000e+00,...,5.936061e-02,0.000000e+00,2.248800e-02,0.000000e+00,0.000000e+00,0.000000,1.915230e-02,7.932888e-02,5.995411e-02,0.000000
Percent Time In Highest Third,2.301965e-02,0.000000e+00,1.453154e-02,0.000000e+00,1.561043e-02,0.000000,2.190644e-02,0.000000e+00,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,2.501607e-02,0.000000e+00,3.847226e-02,0.000000,0.000000e+00,0.000000e+00,2.566169e-02,0.000000
